In [1]:
import open3d as o3d
import numpy as np
import copy
import math

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# 用户选点
def pick_points(pcd):
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    # mesh = o3d.io.read_triangle_mesh("1.ply")
    # mesh.compute_vertex_normals() 
    # 注，同时显示点云和网格可能是不现实的
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()
    return vis.get_picked_points()

# 找圆心
def find_center(p1, p2, p3):
    x1 = p1[0];y1 = p1[1];z1 = p1[2]
    x2 = p2[0];y2 = p2[1];z2 = p2[2]
    x3 = p3[0];y3 = p3[1];z3 = p3[2]
    a1 = (y1*z2 - y2*z1 - y1*z3 + y3*z1 + y2*z3 - y3*z2)
    b1 = -(x1*z2 - x2*z1 - x1*z3 + x3*z1 + x2*z3 - x3*z2)
    c1 = (x1*y2 - x2*y1 - x1*y3 + x3*y1 + x2*y3 - x3*y2)
    d1 = -(x1*y2*z3 - x1*y3*z2 - x2*y1*z3 + x2*y3*z1 + x3*y1*z2 - x3*y2*z1)
    a2 = 2 * (x2 - x1)
    b2 = 2 * (y2 - y1)
    c2 = 2 * (z2 - z1)
    d2 = x1*x1 + y1*y1 + z1*z1 - x2*x2 - y2*y2 - z2*z2
    a3 = 2 * (x3 - x1)
    b3 = 2 * (y3 - y1)
    c3 = 2 * (z3 - z1)
    d3 = x1*x1 + y1*y1 + z1*z1 - x3*x3 - y3*y3 - z3*z3
    x = -(b1*c2*d3 - b1*c3*d2 - b2*c1*d3 + b2*c3*d1 + b3*c1*d2 - b3*c2*d1) / (a1*b2*c3 - a1*b3*c2 - a2*b1*c3 + a2*b3*c1 + a3*b1*c2 - a3*b2*c1)
    y = (a1*c2*d3 - a1*c3*d2 - a2*c1*d3 + a2*c3*d1 + a3*c1*d2 - a3*c2*d1) / (a1*b2*c3 - a1*b3*c2 - a2*b1*c3 + a2*b3*c1 + a3*b1*c2 - a3*b2*c1)
    z = -(a1*b2*d3 - a1*b3*d2 - a2*b1*d3 + a2*b3*d1 + a3*b1*d2 - a3*b2*d1) / (a1*b2*c3 - a1*b3*c2 - a2*b1*c3 + a2*b3*c1 + a3*b1*c2 - a3*b2*c1)
    return x, y, z

# 移动模型
def change_mesh(mesh_first, x, y, z):
    a = [-x, -y, -z]
    mesh_second = copy.deepcopy(mesh_first).translate(tuple(a))
    mesh_second.compute_vertex_normals()
    return mesh_second

# 创建球体标记圆心
def creat_sphere(x, y, z):
    mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.0,
                                                      resolution=100)
    mesh_sphere.compute_vertex_normals()
    mesh_sphere.paint_uniform_color([0.1, 0.1, 0.7])
    a = mesh_sphere.get_center() + [x, y, z]
    print (a)
    mesh_ty = copy.deepcopy(mesh_sphere).translate(tuple(a))
    return mesh_ty

# 找法向量
def find_normal_vector(p1, p2, p3):
    x1 = p1[0];y1 = p1[1];z1 = p1[2]
    x2 = p2[0];y2 = p2[1];z2 = p2[2]
    x3 = p3[0];y3 = p3[1];z3 = p3[2]
    a = (y2 - y1) * (z3 - z1) - (y3 - y1) * (z2 - z1)
    b = (z2 - z1) * (x3 - x1) - (z3 - z1) * (x2 - x1)
    c = (x2 - x1) * (y3 - y1) - (x3 - x1) * (y2 - y1)
    return [a, b, c]

# 移动圆柱体
def change_cylinder(mesh_cylinder1):
    a = - np.asarray(mesh_cylinder1.vertices)[0] + [0, 0, 0]
    mesh_cylinder2 = copy.deepcopy(mesh_cylinder1).translate(tuple(a))
    mesh_cylinder2.compute_vertex_normals()
    R = mesh.get_rotation_matrix_from_xyz((0, np.pi * 1, 0))
    mesh_cylinder = copy.deepcopy(mesh_cylinder2)
    mesh_cylinder.rotate(R, center=point_coordinate)
    return mesh_cylinder

# 判断点是否在模型内
def find_dis(point, mesh):
    mesh2 = copy.deepcopy(mesh)
    mesh2 = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
    scene = o3d.t.geometry.RaycastingScene()
    _ = scene.add_triangles(mesh2)
    query_point = o3d.core.Tensor([point], dtype=o3d.core.Dtype.Float32)
    return scene.compute_signed_distance(query_point)

def find_dis2(point):
    # mesh = copy.deepcopy(mesh)
    query_point = o3d.core.Tensor([point], dtype=o3d.core.Dtype.Float32)
    return scene.compute_signed_distance(query_point)

# 统计该方向上位于模型内部的点的数量
def amount_point(normal_vector, mesh_second):
    length = 0.1
    j = 0
    for i in range(100):
        vector_point = normal_vector * (length * i)
        if find_dis(vector_point, mesh_second) < 0:
            j = j + 1
    return j

# 旋转模型
def rotate_mesh(normal_vector):
    global mesh_second, point_coordinate
    # 向量OB，也就是法向量
    vector_ob = [normal_vector[0], normal_vector[1], normal_vector[2]]

    # 法向量与z轴的夹角
    theta = find_angle(vector_ob, [0, 0, 0], [0, 0, 1])

    # 第一次旋转
    vector_ob2 = [0, np.sin(np.pi * theta), np.cos(np.pi * theta)]
    alpha = find_angle(vector_ob, [0, 0,np.cos(np.pi * theta)], vector_ob2)
    if vector_ob[0] < 0:
        alpha = - alpha
    
    R = mesh_second.get_rotation_matrix_from_xyz((0, 0, np.pi * alpha))
    mesh_third = copy.deepcopy(mesh_second)
    mesh_third.rotate(R, center=point_coordinate)

    # 第二次旋转
    R = mesh_second.get_rotation_matrix_from_xyz((np.pi * theta, 0, 0))
    mesh_fourth = copy.deepcopy(mesh_third)
    mesh_fourth.rotate(R, center=point_coordinate)
    return mesh_fourth
    
# 功能函数，找两点之间距离
def dis(x, y):
    return np.sqrt((x[0] - y[0]) ** 2 + (x[1] - y[1]) ** 2 + (x[2] - y[2]) ** 2)

# 找三个点形成的向量的夹角
# 假定是p2p1向量和p2p3向量之间的夹角
# 返回的是pi的某个倍数
def find_angle(p1, p2, p3):
    l1 = dis(p1, p2); l2 = dis(p2, p3); l3 = dis(p1, p3)
    cos = (l1 ** 2 + l2 ** 2 - l3 ** 2) / (2 * l1 * l2)
    return math.acos(cos)/np.pi

# 找钉子正确位置
def find_true_location(normal_vector, mesh_cylinder):
    global mesh, point_coordinate
    # 向量OB,也就是法向量
    vector_ob = [normal_vector[0], normal_vector[1], normal_vector[2]]
    vector_ob = [vector_ob[0] / vector_ob[2], vector_ob[1] / vector_ob[2], 1]

    # 法向量和圆柱向量的夹角,即向量n和向量OA的夹角
    theta = find_angle(vector_ob, [0, 0, 0], [0, 0, 1])

    # 第一次旋转
    R = mesh.get_rotation_matrix_from_xyz((np.pi * theta, 0, 0))
    mesh_cylinderfirst = copy.deepcopy(mesh_cylinder)
    mesh_cylinderfirst.rotate(R, center=point_coordinate)

    # 第二次旋转
    vector_oa2 = [0, np.sin(np.pi - np.pi * theta) / np.cos(np.pi - np.pi * theta), 1]

    alpha = find_angle(vector_ob, [0, 0, 1], vector_oa2)
    R = mesh.get_rotation_matrix_from_xyz((0, 0, np.pi * alpha))
    mesh_cylindersecond = copy.deepcopy(mesh_cylinderfirst)
    mesh_cylindersecond.rotate(R, center=point_coordinate)

    a = + np.asarray(mesh_cylindersecond.vertices)[0] - np.asarray(mesh_cylindersecond.vertices)[1]
    mesh_cylinderthird = copy.deepcopy(mesh_cylindersecond).translate(tuple(a))
    mesh_cylinderthird.compute_vertex_normals()
    return mesh_cylinderthird

# 改变基座位置
def change_jizuo(mesh_jizuo1):
    a = - mesh_jizuo1.get_center() + [0, 0, 0]
    mesh_jizuo = copy.deepcopy(mesh_jizuo1).translate(tuple(a))
    mesh_jizuo.compute_vertex_normals()
    return mesh_jizuo

# 旋转基座
def rotate_jizuo(mesh_jizuo, direction_vector):
    global mesh, point_coordinate
    
    # 向量OC为基座方向向量
    vector_oc = mesh_jizuo.get_center() - bottom_center
    vector_oc = [vector_oc[0] / vector_oc[2], vector_oc[1] / vector_oc[2], 1]
    print(vector_oc)

    # 向量OD为钉子方向向量
    vector_od = [direction_vector[0] / direction_vector[2], direction_vector[1] / direction_vector[2], 1]
    print(vector_od)

    # 基座方向向量与z轴夹角
    theta = find_angle(vector_oc, [0, 0, 0], [0, 0, 1])

    # 钉子方向向量与z轴夹角
    beta = find_angle(vector_od, [0, 0, 0], [0, 0, 1])

    # 第一次旋转
    vector_oc2 = [0, np.sin(np.pi * theta) / np.cos(np.pi * theta), 1]
    alpha = find_angle(vector_oc, [0, 0, 1], vector_oc2)
    
    R = mesh.get_rotation_matrix_from_xyz((0, 0, - np.pi * alpha))
    mesh_jizuofirst = copy.deepcopy(mesh_jizuo)
    mesh_jizuofirst.rotate(R, center=point_coordinate)

    # 第二次旋转
    R = mesh.get_rotation_matrix_from_xyz((np.pi * theta - np.pi * beta, 0, 0))
    mesh_jizuosecond = copy.deepcopy(mesh_jizuofirst)
    mesh_jizuosecond.rotate(R, center=point_coordinate)

    # 第三次旋转
    vector_oc3 = [0, np.sin(np.pi * beta) / np.cos(np.pi * beta), 1]
    delte = find_angle(vector_oc3, [0, 0, 1], vector_od)
    if vector_od[0] < 0:
        delte = - delte

    R = mesh.get_rotation_matrix_from_xyz((0, 0, - np.pi * delte))
    mesh_jizuothird = copy.deepcopy(mesh_jizuosecond)
    mesh_jizuothird.rotate(R, center=point_coordinate)
    return mesh_jizuothird


In [3]:
# 读入所需点云
# 找时间用pyqt写个小窗口读文件
# 应该是不支持中文路径
path = input()
pcd = o3d.io.read_point_cloud(path)
print (pcd)

# 得到点云的点
value = pcd.points

# 调用选点函数
picked_id_pcd = pick_points(pcd)

# 调用找圆心函数
p1 = value[picked_id_pcd[0]]
p2 = value[picked_id_pcd[1]]
p3 = value[picked_id_pcd[2]]
x, y, z = find_center(p1, p2, p3)
r_circle = np.sqrt((p1[0] - x)**2 + (p1[1] - y)**2 + (p1[2] - z)**2)
print (x, y, z)

# 再次读入所需网格；这样避免了表面重建
mesh_first = o3d.io.read_triangle_mesh(path)
mesh_first.compute_vertex_normals()

# 移动模型到O点
mesh_second = change_mesh(mesh_first, x, y, z)
point_coordinate = (0, 0, 0)
give_point = [p1, p2, p3, point_coordinate]

# 创建球体
mesh_sphere = creat_sphere(0, 0, 0)

# 展示一下
# o3d.visualization.draw_geometries([mesh, mesh_sphere], mesh_show_back_face=True)

# 计算单位法向量
normal_vector_zero = find_normal_vector(p1, p2, p3)
normal_vector_module = (normal_vector_zero[0] **2 + normal_vector_zero[1] **2 + normal_vector_zero[2] **2) **0.5
normal_vector = (np.asarray(normal_vector_zero)) / normal_vector_module
normal_vector_back = normal_vector * (-1)

# 取朝模型内的为法向量
numeber =  amount_point(normal_vector, mesh_second)
numeber_back = amount_point(normal_vector_back, mesh_second)
if numeber_back > numeber:
    normal_vector = normal_vector_back

# 旋转模型，使法向量与z轴重合
mesh = rotate_mesh(normal_vector)

# 创建圆柱
mesh_cylinder1 = o3d.geometry.TriangleMesh.create_cylinder(radius=3.25,
                                                          height=50)

# 改变圆柱位置
mesh_cylinder = change_cylinder(mesh_cylinder1)

# 展示一下
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size = 100)
mesh_frame.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_cylinder, mesh, mesh_sphere, mesh_frame])

mesh2 = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
scene = o3d.t.geometry.RaycastingScene()
_ = scene.add_triangles(mesh2)

PointCloud with 170294 points.
[Open3D INFO] Picked point #58359 (1.0e+02, 8.0, 1.2e+03) to add in queue.
[Open3D INFO] Picked point #46122 (1.1e+02, 20., 1.2e+03) to add in queue.
[Open3D INFO] Picked point #67929 (1.2e+02, 37., 1.2e+03) to add in queue.
[Open3D INFO] Remove picked point #67929 from pick queue.
[Open3D INFO] Picked point #68160 (1.2e+02, 36., 1.2e+03) to add in queue.
108.55989154409072 21.94324907624317 1196.255690888732
[ 3.40307858e-17  1.00093400e-17 -1.11347486e-17]


In [4]:
%%time
# 获取钉子的正确位置
# 目前还在寻找优化的方式。现在大概需要6分钟左右来寻找比较合适的位置
theta1 = 5/5; theta2 = 360/20
location = [0, [], []] # 长度，点的位置，圆柱的位置
for i in range(5):
    for j in range(20):
        # 穷举每个位置的圆柱
        theta_y = 10 + theta1 * i; theta_z = theta2 * j
        R = mesh.get_rotation_matrix_from_xyz((0, theta_y * np.pi / 180, 0))
        mesh_cylinderchange1 = copy.deepcopy(mesh_cylinder)
        mesh_cylinderchange1.rotate(R, center=point_coordinate)
        R = mesh.get_rotation_matrix_from_xyz((0, 0, theta_z * np.pi / 180))
        mesh_cylinderchange = copy.deepcopy(mesh_cylinderchange1)
        mesh_cylinderchange.rotate(R, center=point_coordinate)

        # 对圆柱每个符合条件的位置进行寻找
        length = 0.1
        dis_origin = 100
        pcd2 = mesh_cylinderchange.sample_points_uniformly(number_of_points=200)
        point = np.asarray(pcd2.points)
        point_dis_coordinate = np.array([dis(point[k], point_coordinate) for k in range(200)])
        
        for k in range(200):
            if (point_dis_coordinate[k] >= dis_origin) or (point_dis_coordinate[k] <= 5):
                continue
            judge1 = -1; judge2 = -1
            position_x = np.arange(0, 80, 0.1) + point[k][0]
            position_x = position_x.reshape(-1, 1)
            position_y = np.repeat(point[k][1], 800).reshape(-1, 1)
            position_z = np.repeat(point[k][2], 800).reshape(-1, 1)
            position = np.concatenate((position_x, position_y, position_z),axis=1)
            dis2 = find_dis2(position)
            dis2 = dis2.reshape(-1)
            if (dis2>=0).all():
                judge1 = 1
            
            position_x = np.arange(-80, 0, 0.1) + point[k][0]
            position_x = position_x.reshape(-1, 1)
            position_y = np.repeat(point[k][1], 800).reshape(-1, 1)
            position_z = np.repeat(point[k][2], 800).reshape(-1, 1)
            position = np.concatenate((position_x, position_y, position_z),axis=1)
            dis2 = find_dis2(position)
            dis2 = dis2.reshape(-1)
            x = np.where(dis2<=0)
            if (dis2>=0).all():
                judge2 = 1

            if (judge1 > 0 or judge2 > 0) and (dis_origin > point_dis_coordinate[k]):
                dis_origin = point_dis_coordinate[k]
                know = point[k]
                
        if (dis_origin != 100) and (dis_origin > location[0]):
            location[0] = dis_origin; location[1] = know; location[2] = [i, j]
        print(location)

[26.081462283829982, array([ 1.36240789,  0.43904154, 26.04215356]), [0, 0]]
[26.081462283829982, array([ 1.36240789,  0.43904154, 26.04215356]), [0, 0]]
[26.081462283829982, array([ 1.36240789,  0.43904154, 26.04215356]), [0, 0]]
[26.081462283829982, array([ 1.36240789,  0.43904154, 26.04215356]), [0, 0]]
[26.081462283829982, array([ 1.36240789,  0.43904154, 26.04215356]), [0, 0]]
[26.081462283829982, array([ 1.36240789,  0.43904154, 26.04215356]), [0, 0]]
[26.081462283829982, array([ 1.36240789,  0.43904154, 26.04215356]), [0, 0]]
[26.081462283829982, array([ 1.36240789,  0.43904154, 26.04215356]), [0, 0]]
[26.081462283829982, array([ 1.36240789,  0.43904154, 26.04215356]), [0, 0]]
[26.081462283829982, array([ 1.36240789,  0.43904154, 26.04215356]), [0, 0]]
[26.081462283829982, array([ 1.36240789,  0.43904154, 26.04215356]), [0, 0]]
[26.081462283829982, array([ 1.36240789,  0.43904154, 26.04215356]), [0, 0]]
[26.081462283829982, array([ 1.36240789,  0.43904154, 26.04215356]), [0, 0]]

In [ ]:
# 约为120ms
# 判断不占时间
# 前6行0ns
# find_dis占时间

In [6]:
# 展示一下
R = mesh.get_rotation_matrix_from_xyz((0, (5/5*location[2][0]+10)*np.pi / 180, 0))
mesh_cylinderchange1 = copy.deepcopy(mesh_cylinder)
mesh_cylinderchange1.rotate(R, center=point_coordinate)
R = mesh.get_rotation_matrix_from_xyz((0, 0, (360/20)*location[2][1]*np.pi / 180))
mesh_cylinderchange = copy.deepcopy(mesh_cylinderchange1)
mesh_cylinderchange.rotate(R, center=point_coordinate)
pcd2 = mesh_cylinderchange.sample_points_uniformly(number_of_points=100)
mesh_sphere2 = creat_sphere(location[1][0], location[1][1], location[1][2])
o3d.visualization.draw_geometries([pcd2, mesh, mesh_sphere2, mesh_sphere])

[ 5.46693015 -0.4618951  35.05834784]


In [ ]:
# 寻找界面
%%time
#找到距离最近的三角网格
mesh1 = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
scene = o3d.t.geometry.RaycastingScene()
scene.add_triangles(mesh1)
a=np.array([])

#穷举关节盂面附近的点以寻找三角网格
for i in range(180):
    for j in range(180):
        for k in range(15):
            x=(-20)+0.5*i;y=(-16)+0.5*j;z=(-10)+1*k
            query_point = o3d.core.Tensor([[x,y,z]],dtype=o3d.core.Dtype.Float32)
            ans = scene.compute_closest_points(query_point)
            points=ans['points'].numpy()
            triangle=ans['primitive_ids'][0].item()
            a=np.append(a,triangle)
            a=a.astype(int)

#网格裁剪  
mesh2 = copy.deepcopy(mesh)
mesh2.triangles = o3d.utility.Vector3iVector(
np.asarray(mesh2.triangles)[a])
mesh2.triangle_normals = o3d.utility.Vector3dVector(
np.asarray(mesh2.triangle_normals)[a])
mesh2.paint_uniform_color([0.1, 0.1, 0.7])

#展示一下
o3d.visualization.draw_geometries([mesh2,mesh])

In [10]:
#读入基座文件
mesh_jizuo1 = o3d.io.read_triangle_mesh("jizuo.stl")
mesh_jizuo1.compute_vertex_normals()

#移动基座到球心
mesh_jizuo = change_jizuo(mesh_jizuo1)

o3d.visualization.draw_geometries([mesh_cylinderchange, mesh, mesh_sphere2, mesh_sphere,mesh_jizuo])

#计算基座定向边框的顶点
obb = mesh_jizuo.get_oriented_bounding_box()
m1 = np.asarray(obb.get_box_points())[0]
m2 = np.asarray(obb.get_box_points())[1]
m3 = np.asarray(obb.get_box_points())[2]

#计算基座底面中心
x1, y1, z1 = find_center(m1, m2, m3)
bottom_center = [x1, y1, z1]

#计算钉子方向向量
direction_vector = np.asarray(mesh_cylinderchange.vertices)[0] - np.asarray(mesh_cylinderchange.vertices)[1]
print(direction_vector)

#旋转基座
mesh_jizuonew = rotate_jizuo(mesh_jizuo, direction_vector)

#展示一下
o3d.visualization.draw_geometries([mesh_cylinderchange, mesh, mesh_sphere2, mesh_sphere, mesh_jizuonew])

[34.96822022 35.47529311  4.3274881 ]
[-1.3259628008831732, 0.6798450037280056, 1]
[8.080489050599919, 8.197663927489877, 1]
